In [48]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [49]:
from utils.helpers import read_stock_data
from utils.plot import plot_predictions, plot_loss
from prepro.data_curation import create_preprocessing_pipeline, prepare_data, create_windows
from prepro.feature_eng import add_derived_features
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from post.eval_performance import log_returns_to_price
import os
import numpy as np
import pandas as pd

In [55]:
test = np.random.rand(20,) * 200
test

array([ 71.69709816,  22.88276259,   8.07227465, 158.87217894,
        19.01817877,  70.230998  ,  70.49713894, 138.12247123,
       191.57424793,  36.32231231,  84.58203188,  80.90091258,
        88.34140624, 135.25395221,  55.01168559, 140.75668542,
        27.30681509,  34.98713774,  32.08619067, 184.31853456])

In [56]:
features = add_derived_features(pd.DataFrame(test, columns=['Close']))
features

,return,log_ret,ret_abs,ret_ma_5,ret_ma_20,ret_std_20,sma_5,sma_20,ret_roll_abs_mean_5,ret_roll_abs_mean_20
0,-14.810488,-1.041949,1.041949,-1.092007,-1.092007,0.070794,34.217378,34.217378,1.092007,1.092007
1,150.799904,2.979665,2.979665,0.265217,0.265217,2.351314,65.381079,65.381079,1.721227,1.721227
2,-139.854000,-2.122705,2.122705,-0.331764,-0.331764,2.260824,56.108499,56.108499,1.821596,1.821596
3,51.212819,1.306394,1.306394,-0.004132,-0.004132,2.090504,55.815279,58.462249,1.718556,1.718556
4,0.266141,0.003782,0.003782,0.225038,-0.002813,1.869807,65.338154,60.181519,1.490899,1.432760
5,67.625332,0.672569,0.672569,0.567941,0.093670,1.725875,91.348193,69.924138,1.417023,1.324161
6,53.451777,0.327135,0.327135,0.037435,0.122853,1.599980,97.888607,83.440817,0.886517,1.199533
7,-155.251936,-1.662843,1.662843,0.129407,-0.075558,1.610666,101.349434,78.728966,0.794545,1.251012
8,48.259720,0.845290,0.845290,0.037186,0.016527,1.546219,104.219640,79.261063,0.702324,1.210440
9,-3.681119,-0.044497,0.044497,0.027531,0.010980,1.466987,106.300395,79.397717,0.710467,1.104445


In [57]:
prep_x, targets = prepare_data(
	pd.DataFrame({
		"Close": test.ravel(),
		"Date": pd.date_range(start="2023-01-01", periods=len(test), freq="D")
	}),
	5
)
targets[:,0]

array([-1.04194859,  2.97966467, -2.12270467,  1.30639448,  0.00378235,
        0.67256864,  0.32713469, -1.66284323,  0.84528964, -0.04449675,
        0.08798382,  0.42594521, -0.89960851,  0.93948714])

In [58]:
features

,return,log_ret,ret_abs,ret_ma_5,ret_ma_20,ret_std_20,sma_5,sma_20,ret_roll_abs_mean_5,ret_roll_abs_mean_20
0,-14.810488,-1.041949,1.041949,-1.092007,-1.092007,0.070794,34.217378,34.217378,1.092007,1.092007
1,150.799904,2.979665,2.979665,0.265217,0.265217,2.351314,65.381079,65.381079,1.721227,1.721227
2,-139.854000,-2.122705,2.122705,-0.331764,-0.331764,2.260824,56.108499,56.108499,1.821596,1.821596
3,51.212819,1.306394,1.306394,-0.004132,-0.004132,2.090504,55.815279,58.462249,1.718556,1.718556
4,0.266141,0.003782,0.003782,0.225038,-0.002813,1.869807,65.338154,60.181519,1.490899,1.432760
5,67.625332,0.672569,0.672569,0.567941,0.093670,1.725875,91.348193,69.924138,1.417023,1.324161
6,53.451777,0.327135,0.327135,0.037435,0.122853,1.599980,97.888607,83.440817,0.886517,1.199533
7,-155.251936,-1.662843,1.662843,0.129407,-0.075558,1.610666,101.349434,78.728966,0.794545,1.251012
8,48.259720,0.845290,0.845290,0.037186,0.016527,1.546219,104.219640,79.261063,0.702324,1.210440
9,-3.681119,-0.044497,0.044497,0.027531,0.010980,1.466987,106.300395,79.397717,0.710467,1.104445


In [62]:
re_price = log_returns_to_price(features['log_ret'].values, test[1])
re_price, test

(array([[  8.07227465, 158.87217894,  19.01817877,  70.230998  ,
          70.49713894, 138.12247123, 191.57424793,  36.32231231,
          84.58203188,  80.90091258,  88.34140624, 135.25395221,
          55.01168559, 140.75668542,  27.30681509,  34.98713774,
          32.08619067, 184.31853456]]),
 array([ 71.69709816,  22.88276259,   8.07227465, 158.87217894,
         19.01817877,  70.230998  ,  70.49713894, 138.12247123,
        191.57424793,  36.32231231,  84.58203188,  80.90091258,
         88.34140624, 135.25395221,  55.01168559, 140.75668542,
         27.30681509,  34.98713774,  32.08619067, 184.31853456]))

In [ ]:
test_size = 5
test[-test_size - 1]

np.float64(55.01168559082177)

In [18]:
# Load data
current_stock = "amzn"
file = os.path.join(os.getcwd(), "../archive", "Stocks", f"{current_stock}.us.txt")

data = read_stock_data(file)

print(data.head(), data.shape)


        Date  Open  High   Low  Close    Volume  OpenInt
0 1997-05-16  1.97  1.98  1.71   1.73  14700000        0
1 1997-05-19  1.76  1.77  1.62   1.71   6106800        0
2 1997-05-20  1.73  1.75  1.64   1.64   5467200        0
3 1997-05-21  1.64  1.65  1.38   1.43  18853200        0
4 1997-05-22  1.44  1.45  1.31   1.40  11776800        0 (5153, 7)


In [21]:
X, y = prepare_data(data, horizon=3)
y.shape


(5129, 3)

In [22]:
y

array([[   1.51,    1.51,    1.53],
       [   1.51,    1.53,    1.5 ],
       [   1.53,    1.5 ,    1.51],
       ...,
       [1120.66, 1123.17, 1132.88],
       [1123.17, 1132.88, 1129.13],
       [1132.88, 1129.13, 1125.35]], shape=(5129, 3))

In [15]:
windows, y = create_windows(X, y, window_size=14)
y.shape

(5116, 3)

In [17]:
test = np.ones((y.shape))
foo = pd.DataFrame(
    {
        k: v
        for i in range(y.shape[1])
        for k, v in [(f"foo{i}", y[:, i]), (f"true{i}", test[:, i])]
    }
)
foo

,foo0,true0,foo1,true1,foo2,true2
0,0.139762,1.0,0.148420,1.0,0.246860,1.0
1,0.008658,1.0,0.107098,1.0,-0.004357,1.0
2,0.098440,1.0,-0.013015,1.0,-0.080761,1.0
3,-0.111455,1.0,-0.179201,1.0,-0.048009,1.0
4,-0.067746,1.0,0.063446,1.0,0.017316,1.0
...,...,...,...,...,...,...
5111,-0.008608,1.0,0.007150,1.0,0.015268,1.0
5112,0.015759,1.0,0.023876,1.0,0.026113,1.0
5113,0.008117,1.0,0.010355,1.0,0.018963,1.0
5114,0.002237,1.0,0.010845,1.0,0.007530,1.0


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(data['Close'], label='Close Price')
plt.plot(y, label='MA', )
plt.grid()

In [ ]:
X_scaled = MinMaxScaler().fit_transform(X)
y_scaler = MinMaxScaler(feature_range=(-1, 1))
y = y_scaler.fit_transform(y)
pipeline = create_preprocessing_pipeline(window_size=10)
X_red = pipeline.fit_transform(X_scaled)

In [ ]:
from prepro.feature_eng import calc_return
returns = calc_return(data['Close'])
returns.isna().sum()
returns.shape

In [ ]:
import pandas as pd
from polygon import RESTClient
from dotenv import load_dotenv

load_dotenv("C:\\Users\\pasca\\Documents\\Playground\\AlgoTrading\\local_secrets.env")
api_key = os.getenv("POLYGON_API_KEY")

client = RESTClient(api_key)
today = pd.Timestamp.now().strftime("%Y-%m-%d")
back_dated = (
    pd.Timestamp.now().replace(year=pd.Timestamp.now().year - 3).strftime("%Y-%m-%d")
)
print(back_dated, today)

history = client.get_aggs(
    ticker="AAPL", multiplier=1, timespan="day", from_=back_dated, to=today
)
chart_data = pd.DataFrame(history)
chart_data["date_formatted"] = chart_data["timestamp"].apply(
    lambda x: pd.to_datetime(x, unit="ms")
)
print(chart_data.shape)
chart_data.head()

In [ ]:
chart_data[["close"]]